In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchprof

In [2]:
class myModel1(nn.Module):
    def __init__(self):
        super(myModel1, self).__init__()
        self.choices = nn.ModuleDict({
                '3x3 conv': nn.Conv2d(1, 1, 3),
                '5x5 conv': nn.Conv2d(1, 1, 5),
                '7x7 conv': nn.Conv2d(1, 1, 7)
        })
        
    def forward(self, x):
        x = self.choices['5x5 conv'](x)
        x = self.choices['3x3 conv'](x)
        x = self.choices['7x7 conv'](x)
        return x

In [3]:
class myModel2(nn.Module):
    def __init__(self):
        super(myModel2, self).__init__()
        self.choices = nn.ModuleDict({
                '3x3 conv': nn.Conv2d(1, 1, 3),
                '5x5 conv': nn.Conv2d(1, 1, 5),
                '7x7 conv': nn.Conv2d(1, 1, 7)
        })
        
    def forward(self, x):
        x = self.choices['3x3 conv'](x)
        x = self.choices['3x3 conv'](x)
        x = self.choices['3x3 conv'](x)
        x = self.choices['3x3 conv'](x)
        x = self.choices['3x3 conv'](x)
        return x

In [4]:
class myModel3(nn.Module):
    def __init__(self):
        super(myModel3, self).__init__()
        self.choices = nn.ModuleDict({
                '3x3 conv': nn.Conv2d(1, 1, 3),
                '5x5 conv': nn.Conv2d(1, 1, 5),
                '7x7 conv': nn.Conv2d(1, 1, 7)
        })
        
    def forward(self, x):
        x = self.choices['7x7 conv'](x)
        x = self.choices['3x3 conv'](x)
        x = self.choices['5x5 conv'](x)
        return x

In [5]:
x = torch.Tensor(np.array([[[ 
    [1,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,0,1,1,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,0,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1],
    [0,0,1,1,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1],
    [1,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,0,1,1,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,0,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1],
    [1,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,0,1,1,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,0,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1],
    [0,0,1,1,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1],
    [1,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,0,1,1,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1], 
    [0,0,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1]    
]]]))

x = Variable(x, requires_grad=True)

In [12]:
model1 = myModel1()
model2 = myModel2()
model3 = myModel3()

print("5 -> 3 -> 7")
with torchprof.Profile(model1, use_cuda=True) as prof:
    model1(x)
print(prof.display(show_events = False))
print("-----------------------------------------------------------")


print("3x3 five times")
with torchprof.Profile(model2, use_cuda=True) as prof:
    model2(x)
print(prof.display(show_events = False))
print("-----------------------------------------------------------")

print("7 -> 3 -> 5")
with torchprof.Profile(model3, use_cuda=True) as prof:
    model3(x)
print(prof.display(show_events = False))

5 -> 3 -> 7
Module        | Self CPU total | CPU total | CUDA total
--------------|----------------|-----------|-----------
myModel1      |                |           |           
└── choices   |                |           |           
 ├── 3x3 conv |      130.917us | 469.464us |  467.680us
 ├── 5x5 conv |      244.521us | 894.497us |  893.472us
 └── 7x7 conv |      113.825us | 404.723us |  403.520us

-----------------------------------------------------------
five 3
Module        | Self CPU total | CPU total | CUDA total
--------------|----------------|-----------|-----------
myModel2      |                |           |           
└── choices   |                |           |           
 ├── 3x3 conv |      545.246us |   1.928ms |    1.921ms
 ├── 5x5 conv |        0.000us |   0.000us |    0.000us
 └── 7x7 conv |        0.000us |   0.000us |    0.000us

-----------------------------------------------------------
7 -> 3 -> 5
Module        | Self CPU total | CPU total | CUDA total
-------